# Cleanin algorithm for financial series data based on Barndorff‐Nielsen, Hansen et al. 2009 – Realized kernels in practice

In [2]:
import pandas as pd
import numpy as np


### Pre processing

In [ ]:
data = data[data.notna()]

### Cleaning general data (Trades and Quotes)

In [3]:
def clean_0(serie, origin = 0, trade = False):
    
    """Function to clean trade or quote data from WRDS-Wharton TAQ data set, based on stepts "P1" to "P3" 
     in Barndorff‐Nielsen, Hansen et al. 2009 
    
    Parameters
    ----------
    serie: Dataframe
           Data frame containing all the trade or quote data in the time span 9:30 to 16:00 
        
       
    origin: String
       Select the origin of the trade (e.g. N for NYSE). By default the function takes all the origins
       Exchange on which the trade occurred:
        A – AMEX
        N – NYSE
        B – Boston
        P – Arca
        C – NSX
        T/Q – NASDAQ
        D – NASD ADF and TRF
        X – Philadelphia
        I – ISE
        M – Chicago
        W – CBOE
        Z – BATS
        1 – Nasdaq prints in Nasdaq stocks Aug/Sep 2006 only

    trade = Boolean
        False if the data set corresponds to quote data. True if it is trade data
       
    Returns
    -------
    Dataframe
        Basic cleaned data set
        
        """
    
    
    #P1. Drop observations before 9:30 and after 16:00
    #Already done
    
    #P2. Delete entries with a bid, ask or transaction price equal to zero 
        
    if trade == False:
        clean_data0 = serie[(serie.BID != 0) & (serie.ASK != 0)]
    else:
        clean_data0 = serie[serie.PRICE != 0]

    #P3. Retain entries originating from a single exchange (NYSE in our application). Delete other entries
    
    if origin != 0:
        clean_data0 = clean_data0[clean_data0.EX == origin] 
    else:
        clean_data0 = clean_data0

    return clean_data0     
        
    

### Trade Data Only

In [18]:
def clean_trade(clean_data):
    """Function to clean trade data from WRDS-Wharton TAQ data set, based in stepts "T1" to "T3" 
     in Barndorff‐Nielsen, Hansen et al. 2009
    
    Parameters
    ----------
    clean_data: Dataframe
           Data frame containing data generated by clean_0 function
           
    Returns
    -------
    Dataframe
        Cleaned set for trade data
        
    """
    #T1. Delete entries with corrected trades. (Trades with a Correction Indicator, CORR != 0)
    
    clean_data = clean_data[clean_data.TR_CORR == 0]
    
    #T2. Delete entries with abnormal Sale Condition. (Trades where COND has a letter code, except for ‘E’ and ‘F’).
    
    clean_data["TR_SCOND"].fillna("@", inplace = True)
    
    clean_data = clean_data[(clean_data.TR_SCOND == '@')| 
                            (clean_data.TR_SCOND == "@E")|
                            (clean_data.TR_SCOND == "E")|
                            (clean_data.TR_SCOND == "F")| 
                            (clean_data.TR_SCOND == "@F")]
    
    #clean_data.TR_SCOND.value_counts() # if we want to know how many of each category are in the sample
    time = clean_data.DATE.apply(str) + clean_data.TIME_M
    clean_data['TIME_M'] = pd.to_datetime(time,format= "%Y%m%d%H:%M:%S.%f")
    
    #T3. If multiple transactions have the same time stamp, use the median price.
    clean_data=clean_data.groupby(['TIME_M'], as_index=False).agg({'DATE':'first',
                                                      'SYM_ROOT':'first',
                                                      'TR_SCOND':'first', 
                                                      'SIZE':'first',
                                                      'PRICE':'median'}).sort_values(by=['SYM_ROOT','TIME_M']).reset_index(drop=True)
    
    #T4, is replaced by Q4 since the time stamp of quotes and trades does not coincide
    """Q4 Delete entries for which the mid-quote deviated by more than 10 mean absolute 
    deviations from a rolling centred median (excluding the observation under consideration) 
    of 50 observations (25 observations before and 25 after)"""
    
    price = np.array(clean_data.PRICE)
    med = []
    m_a_d = []
    for i in range(0, len(price)):
        p = np.concatenate((price[i-25:i],price[i+1:i+26]), axis=0)
        med_0 = np.median(p)
        med.append(med_0)
        m_a_d.append(np.mean(np.abs(p - med_0)))
    
    m_a_d = np.array(m_a_d)
    med = np.array(med)
    clean_data = clean_data.drop(clean_data[abs(clean_data.PRICE-med)>10*m_a_d].index)
    
    return clean_data


### Quote data only 

In [5]:
def clean_quote(clean_data):
    """Function to clean quote data from WRDS-Wharton TAQ data set, based in stepts "Q1" to "Q4" 
     in Barndorff‐Nielsen, Hansen et al. 2009
    
    Parameters
    ----------
    clean_data: Dataframe
           Data frame containing data generated by clean_0 function
           
    Returns
    -------
    Dataframe
        Cleaned set for quote data
        
    """
    
    #Q1. When multiple quotes have the same time stamp, we replace all these with a single entry
        #with the median bid and median ask price.
    
    time = clean_data.DATE.apply(str) + clean_data.TIME_M
    #clean_data['TIME_M'] = pd.to_datetime(clean_data.TIME_M,format= "%Y-%m%-d%H:%M:%S.%f")
    clean_data['TIME_M'] = pd.to_datetime(time,format= "%Y%m%d%H:%M:%S.%f")

    clean_data=clean_data.groupby(['TIME_M'], as_index=False).agg({'DATE':'first',
                                                          'SYM_ROOT':'first',
                                                          'QU_SOURCE':'first', 
                                                          'BID':'median',
                                                          'ASK':'median',
                                                          'ASKSIZ':'sum',
                                                          'BIDSIZ':'sum',
                                                          }).sort_values(by=['SYM_ROOT','TIME_M']).reset_index(drop=True)
    
    #Q2. Delete entries for which the spread is negative.
    
    clean_data['SPREAD'] = clean_data.ASK - clean_data.BID
    
    clean_data = clean_data[(clean_data.SPREAD > 0)]
    
    #Q3 Delete entries for which the spread is more that 50 times the median spread on that day
    
    data_day = clean_data.groupby(['DATE'])['SPREAD'].median().reset_index()
    
    clean_data['MEDIAN_SPREAD'] = 0
    
    for i in range(len(data_day)):
        clean_data.MEDIAN_SPREAD.loc[clean_data.DATE == data_day.DATE[0]] = data_day.SPREAD[0]
    
    clean_data = clean_data[clean_data.SPREAD < 50 * clean_data.MEDIAN_SPREAD].copy()
    #Q4 Delete entries for which the mid-quote deviated by more than 10 mean absolute deviations from
    #a rolling centred median (excluding the observation under consideration) 
    #of 50 observations (25 observations before and 25 after)
    
    clean_data['MID_QUOTE'] = 0.5 * (clean_data.ASK + clean_data.BID)
    price = np.array(clean_data.MID_QUOTE)
    med = []
    m_a_d = []
    for i in range(0, len(price)):
        p = np.concatenate((price[i-25:i],price[i+1:i+26]), axis=0)
        med_0 = np.median(p)
        med.append(med_0)
        m_a_d.append(np.mean(np.abs(p - med_0)))
    
    m_a_d = np.array(m_a_d)
    med = np.array(med)
    clean_data = clean_data.drop(clean_data[abs(clean_data.MID_QUOTE-med)>10*m_a_d].index)
    
    return clean_data
        

## Cleaning data

In [7]:
def cleaner(serie, origin = 0, trade = False):
  """Function to clean quote or trade data from WRDS-Wharton TAQ data set, based in stepts "Q1" to "Q4" 
     in Barndorff‐Nielsen, Hansen et al. 2009
    
    Parameters
    ----------
    serie: Dataframe
           Data frame containing data generated from WRDS-Wharton TAQ data set
           
    Returns
    -------
    Dataframe
        Cleaned set for quote or trade data
        
    """

  clean_data0 = clean_0(serie, origin, trade)

  if trade == False:
    clean_data = clean_quote(clean_data0)
  else:
    clean_data = clean_trade(clean_data0)

  return clean_data

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Dataset Quotes Apple 06.05.2010
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/AAPL Apple 06.05.2010 QUOTE.csv')
apple_clean = cleaner(data)
apple_clean.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/apple_cleaned_quote.csv')

In [ ]:
# Dataset Quotes Dow Jones (DIA ETF of DJIA)
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/DIA Dow Jones 05.02.2018 QUOTE.csv')
dia_clean = cleaner(data)
dia_clean.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/djia_cleaned_quote.csv')

In [ ]:
#Dataset Quotes 3M 05.02.2018
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/MMM 3M 05.02.2018 QUOTE.csv')
mmm_clean = cleaner(data)
mmm_clean.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/mmm_cleaned_quote.csv')

In [ ]:
#Dataset Quotes S&P 500 (SPY DTF) 06.05.2010
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/SPY SP 500 06.05.2010 QUOTE.csv')
sp_clean_1 = cleaner(data)
sp_clean_1.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/s&p_cleaned_quote1.csv')

In [ ]:
#Dataset Quotes S&P 500 (SPY DTF) 24.08.2015
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/SPY SP 500 24.08.2015 QUOTE.csv')
sp_clean_2 = cleaner(data)
sp_clean_2.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/s&p_cleaned_quote2.csv')

In [19]:
#Dataset trades S&P 500 (SPY DTF) 06.05.2010
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/SPY 06.05.2010 TRADE.csv')
sp_clean_trade = cleaner(data,trade=True)
sp_clean_trade.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/s&p_cleaned_trade.csv')

In [20]:
#Dataset trades Dow Jones (DIA ETF of DJIA) 05.02.2018
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/DIA Dow Jones 05.02.2018 TRADE.csv')
dia_clean_trade = cleaner(data,trade=True)
dia_clean_trade.to_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/djia_cleaned_trade.csv')